# **Group 18: Exploring Models for Automatically Detecting Climate Change Disinformation**

---
CS 152: Trust and Safety Engineering (Winter 2022)















## ***Exploring Datasets***


---

**Climate-FEVER Dataset**

> In order for our bot to detect climate change disinformation content, we first explored different datasets out there that we can potentially use to build our autodetection models. We found Climate-FEVER dataset on HuggingFace which labels 1,535 real world climate change claims with "SUPPORTS" if there is evidence for the claims, "REFUTES" if evidence refutes the claims, and "NOT ENOUGH INFO" for claims that are ambiguous. In other words, claims those labeled "REFUTES" are climate skeptic claims with no base evidence. And we used the data from this dataset to test our models. You can find the datset here: https://huggingface.co/datasets/climate_fever

> The data is downloadable in a jsonl file, which stores each claim as a json object with "id", "text", and "metadata", where the metadata includes the three labels. In the two models below, we used the claims that are labeled as "REFUTES" since they are the claims with misinfo. Among the 1,535 claims, only about 300 are labeled as "REFUTES" which makes the dataset to be quite small.

**Small database built from online articles**
> We also built a small csv database of known climate change myths claims based on articles such as https://skepticalscience.com/argument.php and https://www.rainforest-alliance.org/everyday-actions/6-claims-made-by-climate-change-skeptics-and-how-to-respond/, which is also used in the two models below.


In [ ]:
!nvidia-smi

## ***Clone GitHub Repository and Install Requirements***

> The Dataset files for the two models are on a separate public repository: https://github.com/memekhant/climate-fever-data



In [ ]:
!git clone https://github.com/memekhant/climate-fever-data

In [ ]:
ls climate-fever-data/

In [ ]:
!pip install transformers

In [ ]:
!pip install -U sentence-transformers

## ***Model I*** 

---

> The Model I explores one of the models discussed in the paper "SGPT: GPT Sentence Embeddings for Semantic Search". The model we explored is a Asymmetric Cross-Encoder Semantic Search Model. Given a single message query, it runs through our list (database) of known climate change disinformation claims and returns a claim in the database that is the most semantically similar match with the message. It returns a score of similarity. The closer the score is to 0, the most similar the claim is.

> We found that the Model I does not really work quite well. The runtime for about 40 claims in our test set is over an hour, and it also could not do the detection effectively. This maybe due to the ambiguous and nuanced nature of climate change disinformation claims that semantic search may be more challenging than general similarity search.

> The paper and GitHub referenced for this Model I can be found on the links below:

*   https://github.com/muennighoff/sgpt
*   https://paperswithcode.com/paper/sgpt-gpt-sentence-embeddings-for-semantic












In [ ]:
# Model I

import json
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from scipy.spatial.distance import cosine
import numpy as np

# Load known climate change data
with open("climate-fever-data/Climate cliams.csv") as file:
    for line in file:
      known_CCD = line.split(',')

def read_jsonl():
  # Set up lists to store the data
  data = []
    
  with open("climate-fever-data/queries.jsonl") as reader:
    for line in reader:
      data.append(json.loads(line))
    

  for item in data:
    if ((item["metadata"]["claim_label"]) == "REFUTES"):
      known_CCD.append(item["text"])
  return known_CCD

docs = read_jsonl()

def get_score(query, docs):
  # Get models - The package will take care of downloading the models automatically
  # For best performance: EleutherAI/gpt-j-6B
  # EleutherAI/gpt-neo-125M
  tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
  model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

  prompt = 'Documents are searched to find matches with the same content.\nThe document "{}" is a good search result for "'

  dict_items = {}
  list_scores = []
  for doc in docs:
      context = prompt.format(doc)

      context_enc = tokenizer.encode(context, add_special_tokens=False)
      continuation_enc = tokenizer.encode(query, add_special_tokens=False)
      # Slice off the last token, as we take its probability from the one before
      model_input = torch.tensor(context_enc+continuation_enc[:-1])
      continuation_len = len(continuation_enc)
      input_len, = model_input.shape

      # [seq_len] -> [seq_len, vocab]
      logprobs = torch.nn.functional.log_softmax(model(model_input)[0], dim=-1).cpu()
      # [seq_len, vocab] -> [continuation_len, vocab]
      logprobs = logprobs[input_len-continuation_len:]
      # Gather the log probabilities of the continuation tokens -> [continuation_len]
      logprobs = torch.gather(logprobs, 1, torch.tensor(continuation_enc).unsqueeze(-1)).squeeze(-1)
      score = torch.sum(logprobs)
      # The higher (closer to 0), the more similar
      # dict_scores.append(f"Document: {doc[:] + ' =>'} Score: {score}")
      dict_items.update({score.item(): doc[:]})
      list_scores.append(score.item())
  index = max(list_scores)
  print(dict_items.get(index))
  return index



# Climate Disinformation
print("These are classified as climate change disinfomation.\n")

print(get_score("The idea that CO2 has magic warming powers is based on Fake Physics.", docs))
print(get_score("what the heck does the ‘climate change consensus’ even mean any more? The definition of climate change consensus is now so fuzzy that leading climate change skeptics are categorizing themselves within the 97%.", docs))
print(get_score("If you believe that sea level rise is a massive problem, you’ve bought into the corporate media’s alarmist narrative.", docs))
print(get_score("Katharine Hayhoe is a Nazi Bitch Whore Climatebecile.", docs))
print(get_score("Environmentalists are basically part of a hysterical cult.", docs))
print(get_score("#RealGreenNotFakeGreen!", docs))
print(get_score("Over thirty years of intense (and extremely expensive) research has totally failed to produce any evidence that human emissions of CO2 are driving climate. CO2 is not a danger to but a benefit for all life on our planet.", docs))
print(get_score("#CO2isGOOD4You!", docs))
print(get_score("Renewable energy sources are not effective or reliable.", docs))
print(get_score("Climate Change is a hoax.", docs)) 
print(get_score("Fossil fuels are not related to global warming.", docs))
print(get_score("There is no scientific consensus on climate change.", docs)) 


# General Climate Info
print("These are general climate change related information but not disinfo\n")

print(get_score("Scientific evidence for warming of the climate system is unequivocal.", docs))
print(get_score("Glaciers are retreating almost everywhere around the world — including in the Alps, Himalayas, Andes, Rockies, Alaska, and Africa.", docs))
print(get_score("Direct observations made on and above Earth’s surface show the planet’s climate is significantly changing. Human activities are the primary driver of those changes.", docs))
print(get_score("Cuts in global greenhouse gas emissions are still far from where they need to be to preserve a livable climate.", docs))
print(get_score("It is undeniable that human activities have warmed the atmosphere, ocean, and land and that widespread and rapid changes in the atmosphere, ocean, cryosphere, and biosphere have occurred.", docs))
print(get_score("There is no question that increased levels of greenhouse gases must cause Earth to warm in response.", docs))
print(get_score("Carbon dioxide from human activity is increasing more than 250 times faster than it did from natural sources after the last Ice Age.", docs))
print(get_score("Collectively, the past eight years are the warmest years since modern recordkeeping began in 1880. This annual temperature data makes up the global temperature record – which tells scientists the planet is warming.", docs))
print(get_score("Too much carbon dioxide in the atmosphere harms earth's plant life.", docs))
print(get_score("Overall, polar bear populations are declining because of global warming.", docs))
print(get_score("The severity of recent droughts and wildfires is driven by the changing climate.", docs))
print(get_score("Recent global warming is caused by humans, not natural events.", docs)) 


# Neither
print("These are general statements unrelated to climate discourse\n")

print(get_score("It's really hot today.", docs))
print(get_score("Polar bears are mentioned in discussions on climate change. I just think they're cute.", docs)) 
print(get_score("Palo Alto generally has nicer weather than Chicago.", docs))
print(get_score("The weather has been really nice since it's almost spring.", docs))
print(get_score("Washington's climate is a lot wetter than Arizona's.", docs))
print(get_score("The air at Yellowstone National Park was clean and crisp!", docs))
print(get_score("I went to see the glaciers in BANFF last weekend; they were so beautiful.", docs))
print(get_score("I love that green and brown sweater combination; this season, green is in.", docs))
print(get_score("Urban development is increasing globally.", docs))
print(get_score("Greenland still have a lot of ice", docs)) 
print(get_score("The sun is so nice today.", docs)) 
print(get_score("Fossil fuels are expensive it is not good for the earth.", docs)) 
print(get_score("My name is John.", docs)) 
print(get_score("Dolphins are cute animals.", docs)) # Not climate disinfo range >30
print(get_score("Ocean Voung is a great poet.", docs)) # Not climate disinfro >30

# Citation

# @article{muennighoff2022sgpt,
#   title={SGPT: GPT Sentence Embeddings for Semantic Search},
#   author={Muennighoff, Niklas},
#   journal={arXiv preprint arXiv:2202.08904},
#   year={2022}
# }

## ***Model II***

---

> For Model II, we used one of the pre-trained models from SentenceTransformers (SBERT) Model. Link: https://www.sbert.net/. Similarly as Model I, given a message query, the message is encoded and compared cosine similarity with encoding of each claim in the database. The model then returns the message with the highest similarity and the score of similarity. We set the threshold of the detection as 0.60, meaning that if the score is above 0.60, then the mesaage is identified as climate change disinformation.

> This model works well in terms of detecting general climate change information. However, it falls short when it comes to detecting climate change disinformation. 





In [ ]:
import json
import re
import torch
from sentence_transformers import SentenceTransformer, util

# Load known climate change data
with open("climate-fever-data/Climate cliams.csv") as file:
    for line in file:
      known_CCD = line.split(',')

def read_jsonl():
  # Set up lists to store the data
  data = []
    
  with open("climate-fever-data/queries.jsonl") as reader:
    for line in reader:
      data.append(json.loads(line))
    

  for item in data:
    if ((item["metadata"]["claim_label"]) == "REFUTES"):
      known_CCD.append(item["text"])
  return known_CCD

docs = read_jsonl()

def get_score(query, docs):
  model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

  list = []
  dict_items = {}
  for doc in docs:
    embedding_1= model.encode(query, convert_to_tensor=True)
    embedding_2 = model.encode(doc, convert_to_tensor=True)
    score = util.pytorch_cos_sim(embedding_1, embedding_2)
    dict_items.update({score.item(): doc[:]})
    list.append(score.item())
  index = max(list)
  print(dict_items.get(index))
  return index

# Climate Disinformation
print("These are classified as climate change disinfomation.\n")

print(get_score("The idea that CO2 has magic warming powers is based on Fake Physics.", docs))
print(get_score("what the heck does the ‘climate change consensus’ even mean any more? The definition of climate change consensus is now so fuzzy that leading climate change skeptics are categorizing themselves within the 97%.", docs))
print(get_score("If you believe that sea level rise is a massive problem, you’ve bought into the corporate media’s alarmist narrative.", docs))
print(get_score("Katharine Hayhoe is a Nazi Bitch Whore Climatebecile.", docs))
print(get_score("Environmentalists are basically part of a hysterical cult.", docs))
print(get_score("#RealGreenNotFakeGreen!", docs))
print(get_score("Over thirty years of intense (and extremely expensive) research has totally failed to produce any evidence that human emissions of CO2 are driving climate. CO2 is not a danger to but a benefit for all life on our planet.", docs))
print(get_score("#CO2isGOOD4You!", docs))
print(get_score("Renewable energy sources are not effective or reliable.", docs))
print(get_score("Climate Change is a hoax.", docs)) 
print(get_score("Fossil fuels are not related to global warming.", docs))
print(get_score("There is no scientific consensus on climate change.", docs)) 


# General Climate Info
print("\nThese are general climate change related information but not disinfo\n")

print(get_score("Scientific evidence for warming of the climate system is unequivocal.", docs))
print(get_score("Glaciers are retreating almost everywhere around the world — including in the Alps, Himalayas, Andes, Rockies, Alaska, and Africa.", docs))
print(get_score("Direct observations made on and above Earth’s surface show the planet’s climate is significantly changing. Human activities are the primary driver of those changes.", docs))
print(get_score("Cuts in global greenhouse gas emissions are still far from where they need to be to preserve a livable climate.", docs))
print(get_score("It is undeniable that human activities have warmed the atmosphere, ocean, and land and that widespread and rapid changes in the atmosphere, ocean, cryosphere, and biosphere have occurred.", docs))
print(get_score("There is no question that increased levels of greenhouse gases must cause Earth to warm in response.", docs))
print(get_score("Carbon dioxide from human activity is increasing more than 250 times faster than it did from natural sources after the last Ice Age.", docs))
print(get_score("Collectively, the past eight years are the warmest years since modern recordkeeping began in 1880. This annual temperature data makes up the global temperature record – which tells scientists the planet is warming.", docs))
print(get_score("Too much carbon dioxide in the atmosphere harms earth's plant life.", docs))
print(get_score("Overall, polar bear populations are declining because of global warming.", docs))
print(get_score("The severity of recent droughts and wildfires is driven by the changing climate.", docs))
print(get_score("Recent global warming is caused by humans, not natural events.", docs)) 


# Neither
print("\nThese are general statements unrelated to climate discourse\n")

print(get_score("It's really hot today.", docs))
print(get_score("Polar bears are mentioned in discussions on climate change. I just think they're cute.", docs)) 
print(get_score("Palo Alto generally has nicer weather than Chicago.", docs))
print(get_score("The weather has been really nice since it's almost spring.", docs))
print(get_score("Washington's climate is a lot wetter than Arizona's.", docs))
print(get_score("The air at Yellowstone National Park was clean and crisp!", docs))
print(get_score("I went to see the glaciers in BANFF last weekend; they were so beautiful.", docs))
print(get_score("I love that green and brown sweater combination; this season, green is in.", docs))
print(get_score("Urban development is increasing globally.", docs))
print(get_score("Greenland is a beautiful snowy country", docs)) 
print(get_score("The sun is so nice today.", docs)) 
print(get_score("Fossil fuels are expensive.", docs)) 
print(get_score("My name is John.", docs)) 
print(get_score("Dolphins are cute animals.", docs)) 
print(get_score("Ocean Voung is a great poet.", docs)) 


# Citation

# @inproceedings{reimers-2019-sentence-bert,
#   title = "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks",
#   author = "Reimers, Nils and Gurevych, Iryna",
#   booktitle = "Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing",
#   month = "11",
#   year = "2019",
#   publisher = "Association for Computational Linguistics",
#   url = "https://arxiv.org/abs/1908.10084",
# }


## ***Other Datasets and Models for Further Research***

**ClimaText Datasets**
> We also came across different training, development, and test datasets for sentence-based climate change topic detection.We were not able to use the datasets on here, but they maybe very useful for further exploration. Link to the datasets: https://www.sustainablefinance.uzh.ch/en/research/climate-fever/climatext.html

**Pre-Trained Climate Change Disinformation Classifier**
> We also found another paper "Computer-assisted classification of contrarian claims about climate change" that had trained AI detection for climate change denial and skepticism claims. The paper was published in Nature Science Reports in December 2021. According to a Washington Post article, their classifier detect climate change misinformation in "five categories: global warming is not happening; human generated greenhouse gases are not causing global warming; climate impacts are not bad; climate solutions will not work; and climate science and scientists are unreliable."
> It seems that this is a very good paper to look at and build on for future implementations.

*   https://www.nature.com/articles/s41598-021-01714-4
*   https://github.com/traviscoan/cards/tree/6214181ef325c6d1270ac5b94761f01759da04be
*   https://www.washingtonpost.com/weather/2021/12/09/climate-change-study-misinformation/

> One of the main challenges of our classifier models both 1 and 2 is the quality of our dataset. Since the dataset itself is small and the claims themselves can be broad or ambiguous, they impact the accuracy of our classifiers. Exploring a building a stronger dataset maybe the next step in the implementation. 










## **Citations**

***For Dataset***

@misc{diggelmann2020climatefever,
      title={CLIMATE-FEVER: A Dataset for Verification of Real-World Climate Claims},
      author={Thomas Diggelmann and Jordan Boyd-Graber and Jannis Bulian and Massimiliano Ciaramita and Markus Leippold},
      year={2020},
      eprint={2012.00614},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}

***For Model I***

@article{muennighoff2022sgpt,
  title={SGPT: GPT Sentence Embeddings for Semantic Search},
  author={Muennighoff, Niklas},
  journal={arXiv preprint arXiv:2202.08904},
  year={2022}
}

***For Model II***

@inproceedings{reimers-2019-sentence-bert,
  title = "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks",
  author = "Reimers, Nils and Gurevych, Iryna",
  booktitle = "Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing",
  month = "11",
  year = "2019",
  publisher = "Association for Computational Linguistics",
  url = "https://arxiv.org/abs/1908.10084",
}



---

Team 18 - Abisola Olawale, Jenny Han, Me Me Khant, Soomin Jun